In [232]:
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [233]:
iris = datasets.load_iris()
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [234]:
X=iris.data[50:] # оставляем только Iris Versicolor Iris Virginica
y=iris.target[50:]
#Y=y.reshape([-1, 1])

In [235]:
y= np.where(y > 1, 1, 0)

In [236]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [237]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [243]:
class LogisticRegression:
    def __init__(self, penalty="l2", gamma=0, fit_intercept=True):
        
        err_msg = "penalty must be 'l1' or 'l2', but got: {}".format(penalty)
        assert penalty in ["l2", "l1"], err_msg
        self.beta = None
        self.gamma = gamma
        self.penalty = penalty
        self.fit_intercept = fit_intercept

    def fit(self, X, y, lr=0.01, tol=1e-7, max_iter=1e7, moment='grad', nester_coef=0.9):
        # convert X to a design matrix if we're fitting an intercept
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]

        grad_prev = 0
        EG = 0 
        l_prev = np.inf
        self.beta = np.random.rand(X.shape[1])
        for _ in range(int(max_iter)):
            y_pred = sigmoid(np.dot(X, self.beta))
            loss = self._NLL(X, y, y_pred)
            if l_prev - loss < tol:
                return
            l_prev = loss
            if moment == 'grad':
                self.beta -= lr * self._NLL_grad(X, y, y_pred)
            elif moment == 'nesterov':
                self.beta -= (nester_coef * grad_prev + (1 - nester_coef) * lr * self._NLL_grad(X, y, y_pred))
                grad_prev = nester_coef * grad_prev + (1 - nester_coef) * lr * self._NLL_grad(X, y, y_pred)
            elif moment == 'RMSProp':
                EG = nester_coef * EG + (1 - nester_coef)*self._NLL_grad(X, y, y_pred)*self._NLL_grad(X, y, y_pred)
                self.beta -=   lr / np.sqrt(EG+tol) * self._NLL_grad(X, y, y_pred)

    def _NLL(self, X, y, y_pred):
        N, M = X.shape
        beta, gamma = self.beta, self.gamma 
        order = 2 if self.penalty == "l2" else 1
        norm_beta = np.linalg.norm(beta, ord=order)
        
        nll = -np.log(y_pred[y == 1]).sum() - np.log(1 - y_pred[y == 0]).sum()
        penalty = (gamma / 2) * norm_beta ** 2 if order == 2 else gamma * norm_beta
        return (penalty + nll) / N

    def _NLL_grad(self, X, y, y_pred):
        N, M = X.shape
        l1norm = lambda x: np.linalg.norm(x, 1)  # noqa: E731
        p, beta, gamma = self.penalty, self.beta, self.gamma
        d_penalty = gamma * beta if p == "l2" else gamma * np.sign(beta)
        return -(np.dot(y - y_pred, X) + d_penalty) / N

    def predict(self, X):
        # convert X to a design matrix if we're fitting an intercept
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        return sigmoid(np.dot(X, self.beta))

Результаты обучения логистической регрессии методом градиентного спуска.

In [239]:
def sigmoid(x):
    #Сигмоид для логистической функции
    return 1 / (1 + np.exp(-x))

In [240]:
def logistic_classifier(X, threshold=0.5):
    #Создаем функцию классифицирующую результаты полученные на основе логистической регрессии с порогом по умолчанию 0,5
    to_classify = model.predict(X)
    return np.where(to_classify >= threshold, 1, 0)

In [241]:
def score(y_pred, y_test):
    #Создаем функцию которая оценивает точность классификатора.
    return np.sum(y_test == y_pred)/ len(y_test)

In [244]:
model=LogisticRegression() #создаем объект логистической регрессии
model.fit(X_train, y_train, moment = 'grad') #обучаем логистическую регрессию методом градиентного спуска

In [245]:
y_pred = logistic_classifier(X_test) #Предсказываем результат для тестовой выборки и класифицируем его

In [246]:
score (y_pred, y_test) #Оцениваем точность классификации

0.9

Результаты обучения логистической регрессии методом моментов Нестерова.

In [247]:
model_nesterov=LogisticRegression()

In [248]:
model_nesterov.fit(X, y, nester_coef=0.9, moment = 'nesterov')

In [249]:
y_pred = logistic_classifier(X) #Предсказываем результат для тестовой выборки и класифицируем его

In [250]:
score (y_pred, y) #Оцениваем точность классификации

0.97

Результаты обучения логистической регрессии методом RMSProp.

In [251]:
model_RMSProp=LogisticRegression()

In [252]:
model_RMSProp.fit(X, y, moment = 'RMSProp')

In [253]:
y_pred = logistic_classifier(X) #Предсказываем результат для тестовой выборки и класифицируем его

In [254]:
score (y_pred, y) #Оцениваем точность классификации

0.97